In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

In [ ]:
data = pd.read_csv('data/credit-card-full.csv', index_col='ID')
X = data.drop(axis=1, columns='default payment next month')
y = data['default payment next month']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75)

In [ ]:
# Create the function
def gbm_grid_search(learning_rate, max_depth, subsample):

	# Create the model
    model = GradientBoostingClassifier(
        learning_rate=learning_rate, 
        max_depth=max_depth, 
        subsample = subsample
    )
    
    # Use the model to make predictions
    predictions = model.fit(X_train, y_train).predict(X_test)
    
    # Return the hyperparameters and score
    return([learn_rate, max_depth, subsample, accuracy_score(y_test, predictions)])

In [ ]:
# Create the relevant lists
results_list = []
learn_rate_list = [.01, .1, .5]
max_depth_list = [2, 4, 6]

# Create the for loop
for learn_rate in learn_rate_list:
    for max_depth in max_depth_list:
        results_list.append(gbm_grid_search(learn_rate,max_depth))

# Print the results
print(results_list)   

In [ ]:
# Create the relevant lists
results_list = []
learn_rate_list = [.01, .1, .5]
max_depth_list = [2, 4, 6]
subsamples = [.4, .6]

# Create the for loop
for learn_rate in learn_rate_list:
    for max_depth in max_depth_list:
        for subsample in subsamples:
            results_list.append(gbm_grid_search(learn_rate, max_depth, subsample))

# Print the results
print(results_list)   

In [ ]:
# Create a Random Forest Classifier with specified criterion
rf_class = RandomForestClassifier(criterion='entropy')

# Create the parameter grid
param_grid = {
    'max_depth': [2, 4, 8, 15], 
    'max_features': ['auto','sqrt']
} 

# Create a GridSearchCV object
grid_rf_class = GridSearchCV(
    estimator=rf_class,
    param_grid=param_grid,
    scoring='roc_auc',
    n_jobs=4,
    cv=5,
    refit=True, return_train_score=True)
print(grid_rf_class)

In [ ]:
grid_rf_class.fit(X_train, y_train).predict(X_test)

In [ ]:
# Read the cv_results property into a dataframe & print it out
cv_results_df = pd.DataFrame(grid_rf_class.cv_results_)
print(cv_results_df)

# Extract and print the column with a dictionary of hyperparameters used
column = cv_results_df.loc[:, ['params']]
print(column)

# Extract and print the row that had the best mean test score
best_row = cv_results_df[cv_results_df['rank_test_score'] == 1 ]
print(best_row)

In [ ]:
# Print out the ROC_AUC score from the best-performing square
best_score = grid_rf_class.best_score_
print(best_score)

# Create a variable from the row related to the best-performing square
cv_results_df = pd.DataFrame(grid_rf_class.cv_results_)
best_row = cv_results_df.loc[[grid_rf_class.best_index_]]
print(best_row)

# Get the n_estimators parameter from the best-performing square and print
best_n_estimators = grid_rf_class.best_params_["n_estimators"]
print(best_n_estimators)

In [13]:
# See what type of object the best_estimator_ property is
print(type(grid_rf_class.best_estimator_))

# Create an array of predictions directly using the best_estimator_ property
predictions = grid_rf_class.best_estimator_.predict(X_test)

# Take a look to confirm it worked, this should be an array of 1's and 0's
print(predictions[0:5])

# Now create a confusion matrix 
print("Confusion Matrix \n", confusion_matrix(y_test, predictions))

# Get the ROC-AUC score
predictions_proba = grid_rf_class.best_estimator_.predict_proba(X_test)[:,1]
print("ROC-AUC Score \n", roc_auc_score(y_test, predictions_proba))

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
[0 1 1 0 0]
Confusion Matrix 
 [[5542  295]
 [1084  579]]
ROC-AUC Score 
 0.7732264708588121
